In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
from logger_utils import custom_sink
from loguru import logger
%matplotlib inline

logger.remove()
logger.add(custom_sink)
functionLogger = logger.bind(format="function_inputs")

train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[1]*train_set_x_orig.shape[2]*train_set_x_orig.shape[3],train_set_x_orig.shape[0])
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[1]*test_set_x_orig.shape[2]*test_set_x_orig.shape[3],test_set_x_orig.shape[0])

train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

def sigmoid(z): return 1/(1+np.exp(-z))

def initialize_with_zeros(dim): return (np.zeros((dim, 1)), 0)



In [ ]:
def propagate(w, b, X, Y):
    functionLogger.bind(w=w,b=b,X=X,Y=Y).debug("Entering propagate with the following args:")
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X) + b)              # compute activation
    functionLogger.bind(wT=w.T,wTX=np.dot(w.T,X),b=b, A=A).debug("Performing Dot product with the following:")
    cost = np.sum(((- np.log(A))*Y + (-np.log(1-A))*(1-Y)))/m  # compute cost
    dw = (np.dot(X,(A-Y).T))/m
    db = (np.sum(A-Y))/m
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    functionLogger.bind(w=w,b=b,X=X,Y=Y).debug("Entering optimize with the following args:")
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - (learning_rate*dw)
        b = b - (learning_rate*db)
        if i % 100 == 0: costs.append(cost)
        if print_cost and i % 100 == 0: print ("Cost after iteration %i: %f" %(i, cost))
    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": db}
    return params, grads, costs

In [3]:
w = np.array([[1.],[2.]])

b = 2

X = np.array([[1.,2.,-1.],[3.,4.,-3.2]])

Y = np.array([[1,0,1]])

In [4]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 500, learning_rate = 0.009)
print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

w = [[-0.35010423]
 [-0.49975908]]
b = 1.9716081506723147
dw = [[ 0.06713807]
 [-0.01446052]]
db = -0.05441432999899335
